![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

# Download using FDSN

SeisMonitor uses MseedDownloader object to download your data according some restrictions defined in a Provider object. 

Let's started importing three important objects to download the data: MseedDownloader,WaveformRestrictions and Provider. And some others from obspy. 

In [1]:
import os
from obspy.core.utcdatetime import UTCDateTime
from obspy.clients.fdsn import Client as FDSNClient
from SeisMonitor.core.objects import WaveformRestrictions,Provider
from SeisMonitor.monitor.downloader.seismonitor import MseedDownloader

Let's define our output folder. Feel free to choose the one you want.

In [2]:
out_download_folder = "../6m"

We have to define **WaveformRestrictions** object in order to initialize Provider object. 

WaveformRestrictions saves all restrictions in your waveform data such as parameters that identify the data, the period to download, sensor or channel preferences, some filters by name or by domain. 

**Let's take a look with the next example !** 

* We are going to download data from [Colombian Seismological network](https://www.fdsn.org/networks/detail/CM/) whose network code is CM.
* We will download the data of all stations whose name begins with the network code VMM and other stations. If we don't know very well the sensor type of each station, we will search data for all locations and for all channels.
* We set the time of interest throught starttime and endtime parameters.
* We want to prioritize broadband seismometer data, otherwise we will look for short period seismoters, acceleremoters, etc.. For this purpose we use location_preferences and channel_preferences(it refers to the instrument type)
* Finally, we want to be sure that the stations are in the Colombian region. So we use filter_domain parameter.


In [3]:
sgc_rest = WaveformRestrictions(network="CM",
                    station="URMC,CLEJA,VILL,PRA,ORTC,GARC,FLO2,CHI,YOT",
                    location="*",
                    channel="*",
                    starttime=UTCDateTime("2019-12-24T19:00:00.000000Z"),
                    endtime=UTCDateTime("2019-12-25T01:00:00.000000Z"),
                    location_preferences=["","00","20","10","40"],
                    channel_preferences=["HH","BH","EH","HN","HL"],
                    filter_networks=[], 
                    filter_stations=[],
                    filter_domain= [-83.101,-64.549,-2.229,14.945],
                    )

Now let's initialize **MseedDownloader** object using **Provider** Object wich in turn uses FDSN Client and WaveformRestrictions.

Keep in mind that you can use more than one Provider. It means you could download data using different WaveformRestrictions. However, the downlaoding time period must be the same.

In [4]:
sgc_client = FDSNClient('http://sismo.sgc.gov.co:8080')
sgc_provider = Provider(sgc_client,sgc_rest)
md = MseedDownloader(providers=[sgc_provider])

MseedDownloader class will download waveform data. Besides it will download xml and json metadata.

In [5]:
json_path = os.path.join(out_download_folder,"stations")
inv,json = md.make_inv_and_json(json_path)

05-15 06:23 [INFO] [SeisMonitor.metadata] running to create inventory and json files
05-15 06:23 [INFO] [SeisMonitor.metadata] Total time of execution: 0.226484 seconds


We reached the end. Now, let's download data using download method.

Take in mind some clarifications in the paramaters:
* **mseed_storage**: Path where to store the waveform files. You could use some key words to set the path for downloading.
* **chunklength_in_sec**: The length of one chunk in seconds. The time between starttime and endtime will be divided into segments of chunklength_in_sec seconds.
* **picker_args**: It's used to know if the stream can be downloaded according to the picker keys. If the the segments given by the length of the stream and overlap parameter are less than batch_size, then no download the stream.
* **n_processor**: Number of threads for downlaoding

In [6]:
mseed_storage = os.path.join(out_download_folder,"downloads/{station}/{network}.{station}.{location}.{channel}__{starttime}__{endtime}.mseed")
md.download(mseed_storage,
            picker_args={"batch_size":100,"overlap":0.3,"length":60},
            chunklength_in_sec=7200,n_processor=None)

05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/CHI/CM.CHI.00.HHE__20191224T185959Z__20191224T205959Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/CHI/CM.CHI.00.HHN__20191224T185959Z__20191224T205959Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/CHI/CM.CHI.00.HHZ__20191224T185959Z__20191224T205959Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/PRA/CM.PRA.00.HHE__20191224T190000Z__20191224T210000Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/PRA/CM.PRA.00.HHN__20191224T190000Z__20191224T210000Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/PRA/CM.PRA.00.HHZ__20191224T190000Z__20191224T210000Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/ORTC/CM.ORTC.00.HHE__20191224T185959Z__20191224T205959Z.mseed  
05-15 06:24 [INFO] [SeisMonitor.Downloader: True] ../6m/downloads/ORTC/CM.ORTC.00.HHN__20191224T185959

# Download using local archive

If you have your own local archive. SeisMonitor could create a LocalClient, and you could use [SDS obspy](https://docs.obspy.org/packages/autogen/obspy.clients.filesystem.sds.Client.html#obspy.clients.filesystem.sds.Client) utilites sucj as [get_waveforms](https://docs.obspy.org/packages/autogen/obspy.clients.filesystem.sds.Client.get_waveforms.html#obspy.clients.filesystem.sds.Client.get_waveforms). Besides you could use Provider utilities as well.

In [7]:
from SeisMonitor.core.client import LocalClient

Just to exemplify, we will use the downloaded data to initialize the **LocalClient**

In [8]:
archive = os.path.join(out_download_folder,"downloads")
my_fmt = "{station}/{network}.{station}.{location}.{channel}__{starttime}__{endtime}.mseed"
local_client = LocalClient(archive,my_fmt)

xml_path = os.path.join(out_download_folder,"stations","inv.xml")
local_provider = Provider(local_client,sgc_rest,xml=xml_path)
print(local_provider)
